In [23]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from scipy.optimize import *
import matplotlib.pyplot as plt

See https://arxiv.org/abs/1304.3061

10.1103/PhysRevX.6.031007

Second Quantized Hamiltonian
\begin{eqnarray*}
    \mathcal{H}(r)=h_0 + \sum_{ij} h_{ij}(r) a^{\dagger}_i a_j +\frac{1}{2} \sum_{ijkl} h_{ijkl}(r) a^{\dagger}_i a^{\dagger}_ja_la_k
\end{eqnarray*}

\begin{eqnarray*}
    h_{ij}(r)=\int{d\mathbf{r}}\,\chi^*_i(\mathbf{r})\left(-\frac{1}{2}\nabla^2-\sum_{a}{\frac{Z_a}{\mathbf{r}_{a,\mathbf{r}}}}\right)\chi_j(\mathbf{r})
\end{eqnarray*}

\begin{eqnarray*}
    h_{ijkl}(r)=\int{d\mathbf{r_1}\,d\mathbf{r_2}}\,\chi^*_i(\mathbf{r_1})\chi^*_j(\mathbf{r_2})r_{1,2}^{-1}\chi_k(\mathbf{r_1})\chi_l(\mathbf{r_2})
\end{eqnarray*}

Jordan-Wigner transformation
\begin{eqnarray*}
    a^{\dagger} = I^{\otimes j-1}\otimes \sigma_{-} \otimes \sigma_{z}^{\otimes N-j}\\
    a = I^{\otimes j-1}\otimes \sigma_{+} \otimes \sigma_{z}^{\otimes N-j}
\end{eqnarray*}

In [24]:
gates = Gates(1)

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [25]:
II = ID.kron(ID)
XX = X.kron(X)
YY = Y.kron(Y)
ZZ = Z.kron(Z)
ZI = Z.kron(ID)
IZ = ID.kron(Z)

sig_iz = [IZ.get()[i,i] for i in range(len(IZ.get()))]
sig_zi = [ZI.get()[i,i] for i in range(len(ZI.get()))]
ZZ.get()

array([[ 1,  0,  0,  0],
       [ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 0,  0,  0,  1]])

In [50]:
def repulsion_energy(Z1=1, Z2=1, r=75e-12):
    Eh = 4.3597447222071e-18 # hartree energy
    ep0 = 8.854187e-12
    e = -1.602176634e-19
    return (1/(4*pi*ep0)*(Z1*Z2*e**2)/r)/Eh

rep_energy = repulsion_energy()
print("Repulsion energy (Eh): %s"%rep_energy)

Repulsion energy (Eh): 0.7055696793059831


In [49]:
g0 = -0.4804
g1 = 0.3435
g2 = -0.4347
g3 = 0.5716
g4 = 0.0910
g5 = 0.0910

H = II*g0 + IZ*g1 + ZI*g2 + ZZ*g3 + XX*g4 + YY*g5

min(scipy.linalg.eig(H.get())[0])+rep_energy

(-1.145629444817661+0j)

In [38]:
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_2q(q1, q2, params):
    q2.CNOT(q1)
    q1.RY(params[0])
    q2.RY(params[1])
    q1.CNOT(q2)
    q1.RY(params[2])
    q2.RY(params[3])
    q2.CNOT(q1)
    q1.RY(params[4])
    q2.RY(params[5])

In [39]:
def expectation_2q(params):
    logicQuBit  = LogicQuBit(2)
    q1 = Qubit()
    q2 = Qubit()

    ansatz_2q(q1,q2,params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_2q, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum.fun+rep_energy)

-1.145629444817662


In [40]:
def expectation_value(measurements, base = np.array([1,-1,-1,1])):
    probabilities = np.array(measurements)
    expectation = np.sum(base * probabilities)
    return expectation

def sigma_xx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_yy(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_zz(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
          
    result = logicQuBit.Measure([q1,q2])
    zz = expectation_value(result)
    iz = expectation_value(result, sig_iz) # [zz, iz] = 0
    zi = expectation_value(result, sig_zi) # [zz, zi] = 0
    return zz, iz, zi

def expectation_energy(params):
    xx =  sigma_xx(params)
    yy =  sigma_yy(params)
    zz, iz, zi =  sigma_zz(params)
    
    result = g0 + g1*iz + g2*zi + g3*zz + g4*xx + g5*yy
    return result

In [41]:
minimum = minimize(expectation_energy, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum.fun+rep_energy)

-1.1456294448176618


In [42]:
def gradient(params, evaluate):
    n_params = params.shape[0]
    shift = pi/2
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [43]:
params = np.random.uniform(-np.pi, np.pi, 6)
last_params = np.zeros(6)

In [44]:
lr = 0.1
err = 1
while err > 1e-5:
    grad = gradient(params, expectation_energy)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

/tmp/ipykernel_3773/1088512639.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


0.8036464520471593
0.066507202474833
0.0650056254104705
0.06317151231647644
0.06091693993038949
0.05811410585290627
0.05457926482233477
0.05005001831855277
0.044152640168749924
0.03635451465793427
0.025895344668580733
0.011692466630882914
0.007769389385788472
0.034494421125069646
0.07072792273707304
0.11772771845743873
0.17236247800167742
0.2216826159166947
0.243294660336585
0.22441170862483945
0.18033502618255065
0.1359246097373531
0.10242284608909788
0.0798027805608332
0.06496061081646312
0.05509308699947513
0.04828391929694614
0.04334248948401881
0.03955883451480463
0.03651667177241848
0.033972086248421046
0.03178015374690006
0.029852259476651183
0.028131865310179943
0.026580948989635464
0.025172460810486658
0.023886115252691575
0.022706011815177576
0.021619255713053308
0.020615127545080138
0.019684558589338944
0.018819780558981994
0.018014078684707502
0.017261608965480534
0.0165572574733992
0.015896528755417538
0.015275455375363296
0.01469052344427561
0.01413861061991259
0.01361693

In [45]:
energy = expectation_energy(params)
energy = energy + rep_energy
print(energy)

(-1.1455966226569918+0j)
